## Import modules

In [2]:
import csv
import urllib.request
import json 
import time
import os
import pandas as pd
from html.parser import HTMLParser
import re
import ast
import decimal
import ssl
import sys
import numpy as np

In [3]:
# auto-generate the current time in 'YYYYMM' format
actionDate = time.strftime('%Y%m%d')

In [ ]:
import urllib.request
import json
import ssl

# Specify the CKAN portal URL you want to harvest from
portalURL = "https://gisdata.mn.gov/"

# Construct the API URL for package search
packageURL = portalURL + 'api/3/action/package_search'

# Specify the number of items per page
items_per_page = 10

# Initialize variables for pagination
start = 0
total_results = 0

# List to store all metadata
all_metadata = []

# Request metadata in paginated manner
while True:
    # Construct the API request URL with pagination parameters
    api_request_url = f"{packageURL}?start={start}&rows={items_per_page}"
    
    # Request metadata
    context = ssl._create_unverified_context()
    response = urllib.request.urlopen(api_request_url, context=context).read()
    response_json = json.loads(response.decode('utf-8'))
    
    # Extract metadata from the response
    metadata = response_json['result']['results']
    all_metadata.extend(metadata)
    
    # Update pagination variables
    start += items_per_page
    total_results = response_json['result']['count']
    
    # Break the loop if we have collected all items
    if start >= total_results:
        break

# Save the metadata to a local JSON file on your desktop
desktop_path = ""  # Replace with your desktop path
output_filename = "ckan_metadata.json"
output_path = desktop_path + output_filename

with open(output_path, "w") as json_file:
    json.dump(all_metadata, json_file, indent=4)

print(f"Metadata for {total_results} items saved to {output_path}")

In [5]:
import pandas as pd

# # Specify the path to the JSON file
output_filename = "ckan_metadata.json"  # Update with the correct path

# Read the JSON file into a DataFrame
df = pd.read_json(output_filename)

# Display the first few rows of the DataFrame
# print(df.head())

In [58]:
# Function to process the resources
def process_resources(resources):
    result = {}
    for resource in resources:
        resource_type = resource["resource_type"]
        url = resource["url"]
        result[resource_type] = url
    return pd.Series(result)

# Apply the function to create the resource columns
resource_columns = df['resources'].apply(process_resources)

# Flatten the nested dictionaries within the "extras" array
def flatten_extras(extras):
    flattened = {}
    for item in extras:
        key = item["key"]
        value = item["value"]
        flattened[key] = value
    return flattened

df["flattened_extras"] = df["extras"].apply(flatten_extras)

# Expand the flattened_extras dictionary columns into separate columns
flattened_extras_df = df["flattened_extras"].apply(pd.Series)

# Flatten the "groups" dictionaries within the "groups" array
def flatten_groups(groups):
    if isinstance(groups, list):
        return "|".join([group["display_name"] for group in groups])
    return ""

df["group_titles"] = df["groups"].apply(flatten_groups)

# Flatten the "tags" dictionaries within the "tags" array
def flatten_tags(tags):
    if isinstance(tags, list):
        return "|".join([tag["display_name"] for tag in tags])
    return ""
df["tag_titles"] = df["tags"].apply(flatten_tags)

# Extract the "title" value from the "organization" dictionary
df["organization_title"] = df["organization"].apply(lambda x: x["title"] if isinstance(x, dict) else "")

# Select the columns you want to keep from the original DataFrame
selected_columns = [
    "title",  
    "id", 
    "name", 
    "notes"
]

# Select the columns you want to keep from the flattened_extras DataFrame
selected_extras_columns = [
    "dsAccessConst", 
    "dsCurrentRef", 
    "dsMetadataUrl", 
    "dsModifiedDate", 
    "dsOriginator", 
    "dsPeriodOfContent", 
    "dsPurpose", 
    "gdrsDsGuid", 
    "spatial"
]

# Combine the selected columns from the original, flattened_extras, and resource_columns DataFrames
selected_df = pd.concat([df[selected_columns], flattened_extras_df[selected_extras_columns], df["group_titles"], df["organization_title"], df["tag_titles"], resource_columns], axis=1)

# Specify the path for the CSV file
csv_file_path = actionDate + "_05a-01.csv"  # Update with the desired path

# Write the selected DataFrame to a CSV file
selected_df.to_csv(csv_file_path, index=False)

print("Selected columns, flattened extras, group titles, tag titles, and resources have been written to CSV.")

Selected columns, flattened extras, group titles, tag titles, and resources have been written to CSV.


In [59]:
counties_in_minnesota = [
    'Aitkin', 'Anoka', 'Becker', 'Beltrami', 'Benton', 'Big Stone',
    'Blue Earth', 'Brown', 'Carlton', 'Carver', 'Cass', 'Chippewa',
    'Chisago', 'Clay', 'Clearwater', 'Cook', 'Cottonwood', 'Crow Wing',
    'Dakota', 'Dodge', 'Douglas', 'Faribault', 'Fillmore', 'Freeborn',
    'Goodhue', 'Grant', 'Hennepin', 'Houston', 'Hubbard', 'Isanti',
    'Itasca', 'Jackson', 'Kanabec', 'Kandiyohi', 'Kittson', 'Koochiching',
    'Lac qui Parle', 'Lake', 'Lake of the Woods', 'Le Sueur', 'Lincoln',
    'Lyon', 'Mahnomen', 'Marshall', 'Martin', 'McLeod', 'Meeker',
    'Mille Lacs', 'Morrison', 'Mower', 'Murray', 'Nicollet', 'Nobles',
    'Norman', 'Olmsted', 'Otter Tail', 'Pennington', 'Pine', 'Pipestone',
    'Polk', 'Pope', 'Ramsey', 'Red Lake', 'Redwood', 'Renville',
    'Rice', 'Rock', 'Roseau', 'St. Louis', 'Scott', 'Sherburne',
    'Sibley', 'Stearns', 'Steele', 'Stevens', 'Swift', 'Todd',
    'Traverse', 'Wabasha', 'Wadena', 'Waseca', 'Washington', 'Watonwan',
    'Wilkin', 'Winona', 'Wright', 'Yellow Medicine'
]

cities_in_minnesota = [
    'Minneapolis', 'St. Paul', 'Rochester', 'Duluth', 'Bloomington',
    'Brooklyn Park', 'Plymouth', 'Woodbury', 'St. Cloud', 'Eagan',
    'Maple Grove', 'Eden Prairie', 'Coon Rapids', 'Burnsville', 'Blaine',
    'Lakeville', 'Minnetonka', 'Apple Valley', 'Edina', 'St. Louis Park','Twin Cities'
]

def transform_with_date(alt_title, date):
    # Transform the title with the existing logic
    transformed_title = transform_title(alt_title)

    # Check if date is not missing and is a string
    if isinstance(date, str) and '/' in date:
        # Extract the year from the date string
        year = date.split('/')[-1]

        # Append the year in curly brackets
        transformed_title += f" {{{year}}}"

    return transformed_title



def transform_title(alt_title):
    bracket_content = None
    
    # 1. Look for ", {county} County, Minnesota" and replace with "[Minnesota--{county} County]"
    for county in counties_in_minnesota:
        if f", {county} County, Minnesota" in alt_title:
            bracket_content = f"[Minnesota--{county} County]"
            alt_title = re.sub(f", {county} County, Minnesota", "", alt_title, 1)

    # 2. Look for "{city}, Minnesota" and replace with "[Minnesota--{city}]"
    if not bracket_content:
        for city in cities_in_minnesota:
            if f"{city}, Minnesota" in alt_title:
                bracket_content = f"[Minnesota--{city}]"
                alt_title = re.sub(f"{city}, Minnesota", "", alt_title, 1)

    # 3. Look for variations of just the state of Minnesota
    if not bracket_content:
        variations = [r", Minnesota", r"For Minnesota", r"Of Minnesota", r"In Minnesota"]
        for variation in variations:
            if variation.lower() in alt_title.lower():
                bracket_content = "[Minnesota]"
                alt_title = re.sub(variation, "", alt_title, flags=re.IGNORECASE)

    # 4. For everything else, add ["Minnesota"] to the end
    if not bracket_content:
        bracket_content = "[Minnesota]"

    # Append bracketed content to the end of the title
    alt_title = f"{alt_title.strip()} {bracket_content}"

    return alt_title

selected_df['New_Title'] = selected_df.apply(lambda row: transform_with_date(row['title'], row['dsPeriodOfContent']), axis=1)


# selected_df['New_Title'] = selected_df['title'].apply(transform_title)

In [60]:
def get_date_range(date):
    if pd.isnull(date) or '/' not in str(date):  # Check for NaN or incorrect format
        return None
    year = str(date).split('/')[-1]  # Extract the year from the date string
    return f"{year}-{year}"  # Format the date range

# Apply the function to the 'dsPeriodOfContent' column and create a new 'Date Range' column
selected_df['Date Range'] = selected_df['dsPeriodOfContent'].apply(get_date_range)


In [62]:
def get_spatial_coverage(originator):
    if pd.isnull(originator): # Check for NaN
        return "Minnesota"

    # Check for counties
    for county in counties_in_minnesota:
        if county in originator:
            return f"Minnesota--{county} County|Minnesota"

    # Check for specific organizations
    if "Metropolitan Council" in originator or "MetroGIS" in originator:
        return "Minnesota--Anoka County|Minnesota--Carver County|Minnesota--Dakota County|Minnesota--Hennepin County|Minnesota--Ramsey County|Minnesota--Scott County|Minnesota--Washington County|Minnesota--Twin Cities Metropolitan Area|Minnesota"
    elif "Metropolitan Emergency Services Board" in originator:
        return "Minnesota--Anoka County|Minnesota--Carver County|Minnesota--Chisago County|Minnesota--Dakota County|Minnesota--Hennepin County|Minnesota--Isanti County|Minnesota--Ramsey County|Minnesota--Scott County|Minnesota--Sherburne County|Minnesota--Washington County|Minnesota--Twin Cities Metropolitan Area|Minnesota"
    
    return "Minnesota"

# Apply the function to the 'dsOriginator' column and create a new 'Spatial Coverage' column
selected_df['Spatial Coverage'] = selected_df['dsOriginator'].apply(get_spatial_coverage)


In [67]:
def get_bounding_box(wkt_str):
    # Default bounding box for Minnesota
    default_bounding_box = "-97.2392,43.4994,-89.4919,49.3845"

    # Check if the value is missing or not a string
    if wkt_str is None or not isinstance(wkt_str, str):
        return default_bounding_box

    try:
        # Parse the string as JSON
        wkt_json = json.loads(wkt_str)

        # Extract the coordinates
        coordinates = wkt_json["coordinates"][0]

        # Calculate the bounding box
        west = min(coord[0] for coord in coordinates)
        south = min(coord[1] for coord in coordinates)
        east = max(coord[0] for coord in coordinates)
        north = max(coord[1] for coord in coordinates)

        # Return the bounding box in the required format
        return f"{west},{south},{east},{north}"

    except (json.JSONDecodeError, KeyError, TypeError):
        return default_bounding_box

# Apply the function to the 'spatial' column and create a new 'Bounding Box' column
selected_df['Bounding Box'] = selected_df['spatial'].apply(get_bounding_box)




In [77]:
# Define the conversion mappings from old values to new values
theme_mapping = {

    "Biota": "Biology",
    "Geoscientific": "Geology",
    "Imagery + Basemaps": "Imagery|Land Cover",
    "Planning + Cadastre": "Property",
    "Utilities + Communication": "Utilities"
    
    # all the other values can be copied as is
}


# Function to apply the mapping and join the values back together
def convert_and_join(row):
    group_values = row['group_titles']
    if pd.notna(group_values):  # Check for NaN before splitting
        group_values = group_values.split('|')
        converted_values = []
        for value in group_values:
            if value in theme_mapping:
                converted_values.append(theme_mapping[value])
            else:  # If the value is not in theme_mapping, append it as-is
                converted_values.append(value)
        return '|'.join(converted_values)
    else:
        return ''  # Return an empty string if the value is NaN

# Apply the mapping and create the new "Theme" column
selected_df['Theme'] = selected_df.apply(convert_and_join, axis=1)



In [78]:
# Write the selected DataFrame to a CSV file
selected_df.to_csv(csv_file_path, index=False)